# Train CIFAR with the `policy` module

Let's install the latest version of Poutyne (if it's not already) and import all the needed packages.

In [1]:
#%pip install --upgrade poutyne

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.models import resnet18

from poutyne import Model, OptimizerPolicy, one_cycle_phases

## Training constant
But first, let's set the training constants, the CUDA device used for training if one is present, we set the batch size (i.e. the number of elements to see before updating the model) and the number of epochs (i.e. the number of times we see the full dataset).

In [2]:
cuda_device = 0
device = torch.device("cuda:%d" % cuda_device if torch.cuda.is_available() else "cpu")

batch_size = 1024
epochs = 5

# Load the data

In [3]:
_mean = [0.485, 0.456, 0.406]
_std = [0.229, 0.224, 0.225]


train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(.3, .3, .3),
    transforms.ToTensor(),
    transforms.Normalize(_mean, _std),
])
val_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(_mean, _std),
])

In [4]:
root = "datasets"
train_ds = datasets.CIFAR10(root, train=True, transform=train_transform, download=True)
val_ds = datasets.CIFAR10(root, train=False, transform=val_transform, download=True)

Files already downloaded and verified
Files already downloaded and verified


In [5]:
train_dl = DataLoader(
    train_ds,
    batch_size=batch_size,
    shuffle=True,
    num_workers=8
)
val_dl = DataLoader(
    val_ds,
    batch_size=batch_size,
    shuffle=False,
    num_workers=8
)

# The model
We'll train a simple `resNet-18` network.
This takes a while without GPU but is pretty quick with GPU.

In [6]:
def get_module():
    model = resnet18(pretrained=False)
    model.avgpool = nn.AdaptiveAvgPool2d(1)
    model.fc = nn.Linear(512, 10)
    return model


# Training without the `policies` module

In [7]:
pytorch_network = get_module()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(pytorch_network.parameters(), lr=0.01)

model = Model(
    pytorch_network,
    optimizer,
    criterion,
    batch_metrics=["acc"],
    device=device,
)

history = model.fit_generator(
    train_dl,
    val_dl,
    epochs=epochs,
)

Epoch: 1/5 Step: 49/49 100.00% |█████████████████████████|10.97s loss: 2.114963 acc: 23.278000 val_loss: 1.864067 val_acc: 32.720000
Epoch: 2/5 Step: 49/49 100.00% |█████████████████████████|10.26s loss: 1.798623 acc: 34.332000 val_loss: 1.666151 val_acc: 39.590000
Epoch: 3/5 Step: 49/49 100.00% |█████████████████████████|10.25s loss: 1.646562 acc: 39.752000 val_loss: 1.557361 val_acc: 42.800000
Epoch: 4/5 Step: 49/49 100.00% |█████████████████████████|10.30s loss: 1.556004 acc: 43.340000 val_loss: 1.495678 val_acc: 45.790000
Epoch: 5/5 Step: 49/49 100.00% |█████████████████████████|10.26s loss: 1.492378 acc: 45.702000 val_loss: 1.441630 val_acc: 47.500000


# Training with the `policies` module

In [8]:
steps_per_epoch = len(train_dl)
steps_per_epoch

49

In [9]:
pytorch_network = get_module()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(pytorch_network.parameters(), lr=0.01)

model = Model(
    pytorch_network,
    optimizer,
    criterion,
    batch_metrics=["acc"],
    device=device,
)

policy = OptimizerPolicy(
    one_cycle_phases(epochs * steps_per_epoch, lr=(0.01, 0.1, 0.008)),
)
history = model.fit_generator(
    train_dl,
    val_dl,
    epochs=epochs,
    callbacks=[policy],
)

Epoch: 1/5 Step: 49/49 100.00% |█████████████████████████|10.32s loss: 1.843946 acc: 33.514000 val_loss: 1.542911 val_acc: 45.300000
Epoch: 2/5 Step: 49/49 100.00% |█████████████████████████|10.13s loss: 1.396645 acc: 49.988000 val_loss: 1.287964 val_acc: 55.170000
Epoch: 3/5 Step: 49/49 100.00% |█████████████████████████|10.44s loss: 1.163528 acc: 58.790000 val_loss: 1.087473 val_acc: 61.030000
Epoch: 4/5 Step: 49/49 100.00% |█████████████████████████|10.24s loss: 0.973432 acc: 65.746000 val_loss: 0.998768 val_acc: 65.050000
Epoch: 5/5 Step: 49/49 100.00% |█████████████████████████|10.31s loss: 0.815666 acc: 71.212000 val_loss: 0.886250 val_acc: 69.210000
